In [1]:
from counterfactuals.explainers import Fimap, Cadex, ECE
from counterfactuals.constraints import OneHot, Freeze, ValueMonotonicity, ValueNominal
from counterfactuals.visualization import show, compare
from data import GermanData
from tensorflow import keras
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)

2022-01-19 20:00:31.260018: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-01-19 20:00:31.260042: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')

2022-01-19 20:00:32.832467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: 
2022-01-19 20:00:32.832495: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-19 20:00:32.832513: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2022-01-19 20:00:32.832717: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
german_data.input.sample(5)

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
747.0,12.0,1274.0,3.0,1.0,37.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
836.0,12.0,886.0,4.0,2.0,21.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
697.0,12.0,926.0,1.0,2.0,38.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
835.0,12.0,1082.0,4.0,4.0,48.0,2.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
932.0,9.0,1224.0,3.0,1.0,30.0,2.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [4]:
X_test = german_data.input.iloc[0]
X_test

duration                                                      6.0
credit                                                     1169.0
installment_percent                                           4.0
residence_duration                                            4.0
age                                                          67.0
existing_credits                                              2.0
people_maintained                                             1.0
account_status_0..200 DM                                      0.0
account_status_< 0 DM                                         1.0
account_status_>= 200 DM                                      0.0
account_status_no checking account                            0.0
credit_history_all credits at this bank paid back duly        0.0
credit_history_critical account                               1.0
credit_history_delay in paying off in the past                0.0
credit_history_existing credits paid back duly till now       0.0
credit_his

In [5]:
X_test_scaled = german_data.scale(X_test)
X_test_scaled

duration                                                  -1.225018
credit                                                    -0.748395
installment_percent                                        0.923195
residence_duration                                         1.040096
age                                                        2.744969
existing_credits                                           1.027823
people_maintained                                         -0.444528
account_status_0..200 DM                                  -0.623610
account_status_< 0 DM                                      1.623688
account_status_>= 200 DM                                  -0.249834
account_status_no checking account                        -0.793302
credit_history_all credits at this bank paid back duly    -0.229416
credit_history_critical account                            1.579066
credit_history_delay in paying off in the past            -0.307232
credit_history_existing credits paid back duly t

In [6]:
model.predict(np.expand_dims(X_test_scaled, axis=0))

array([[0.00851224, 0.99148774]], dtype=float32)

In [7]:
cadex = Cadex(model)
cf = cadex.generate(X_test_scaled)

In [8]:
model.predict(cf)

array([[0.5001611 , 0.49983895]], dtype=float32)

In [9]:
show(X_test_scaled, cf)

,X,X',change
duration,-1.225018,0.796423,2.021442
account_status_< 0 DM,1.623688,3.645130,2.021441
account_status_no checking account,-0.793302,-2.814743,-2.021441
debtors_guarantor,-0.246998,-2.268439,-2.021441
foreign_yes,0.193942,2.215383,2.021441


In [10]:
show(X_test, german_data.unscale(cf))

,X,X',change
duration,6.0,3.057253e+01,2.457253e+01
account_status_< 0 DM,1.0,1.902602e+00,9.026024e-01
account_status_no checking account,0.0,-9.842176e-01,-9.842176e-01
debtors_guarantor,0.0,-4.705824e-01,-4.705824e-01
job_unemployed/unskilled - non-resident,0.0,-3.469447e-18,-3.469447e-18
job_unskilled - resident,0.0,-2.775558e-17,-2.775558e-17
foreign_yes,1.0,1.377831e+00,3.778307e-01


In [11]:
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(X_test)

In [12]:
show(X_test, cf)

,X,X',change
duration,6.0,3.062998e+01,2.462998e+01
account_status_< 0 DM,1.0,1.904713e+00,9.047126e-01
account_status_no checking account,0.0,-9.865187e-01,-9.865187e-01
debtors_guarantor,0.0,-4.716826e-01,-4.716826e-01
job_unemployed/unskilled - non-resident,0.0,-3.469447e-18,-3.469447e-18
job_unskilled - resident,0.0,-2.775558e-17,-2.775558e-17
foreign_yes,1.0,1.378714e+00,3.787141e-01


In [13]:
show(X_test_scaled, german_data.scale(cf))

,X,X',change
duration,-1.225018,0.801150,2.026168
account_status_< 0 DM,1.623688,3.649856,2.026167
account_status_no checking account,-0.793302,-2.819469,-2.026167
debtors_guarantor,-0.246998,-2.273165,-2.026167
foreign_yes,0.193942,2.220109,2.026167


In [14]:
model.predict(german_data.scale(cf))

array([[0.5029423 , 0.49705765]], dtype=float32)